# 声明：内容为学习笔记，仅供学习交流

# LDA模型应用：一眼看穿邮件

邮件，跑一把LDA，看看平时都在聊什么。


In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv("data/data78204/HillaryEmails.csv")
# 原邮件数据中有很多Nan的值，直接扔了。
df = df[['Id','ExtractedBodyText']].dropna()

### 文本预处理：


针对邮件内容，写一组正则表达式：

（不熟悉正则表达式的，直接百度关键词，可以看到一大张Regex规则表）

In [3]:
def clean_email_text(text):
    text = text.replace('\n'," ") #新行，我们是不需要的
    text = re.sub(r"-", " ", text) #把 "-" 的两个单词，分开。（比如：pre-processing ==> pre processing）
    text = re.sub(r"\d+/\d+/\d+", "", text) #日期，对主体模型没什么意义
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text) #时间，没意义
    text = re.sub(r"[\w]+@[\.\w]+", "", text) #邮件地址，没意义
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text) #网址，没意义
    pure_text = ''
    # 以防还有其他特殊字符（数字）等等，我们直接把他们loop一遍，过滤掉
    for letter in text:
        # 只留下字母和空格
        if letter.isalpha() or letter==' ':
            pure_text += letter
    # 再把那些去除特殊字符后落单的单词，直接排除。
    # 我们就只剩下有意义的单词了。
    text = ' '.join(word for word in pure_text.split() if len(word)>1)
    return text

In [4]:
docs = df['ExtractedBodyText']
docs = docs.apply(lambda s: clean_email_text(s))  

In [5]:
# 查看格式
docs.head(1).values

array(['Thursday March PM Latest How Syria is aiding Qaddafi and more Sid hrc memo syria aiding libya docx hrc memo syria aiding libya docx March For Hillary'],
      dtype=object)

In [6]:
#读取邮件内容
doclist = docs.values

### LDA模型构建：

用Gensim来做一次模型构建

整出来的一大波文本数据
```
[[一条邮件字符串]，[另一条邮件字符串], ...]
```

转化成Gensim认可的语料库形式：

```
[[一，条，邮件，在，这里],[第，二，条，邮件，在，这里],[今天，天气，肿么，样],...]
```

引入库：

In [7]:
!pip install gensim

Looking in indexes: https://mirror.baidu.com/pypi/simple/


In [8]:
!pip install -U numpy==1.19.2

Looking in indexes: https://mirror.baidu.com/pypi/simple/
Requirement already up-to-date: numpy==1.19.2 in /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages (1.19.2)


In [9]:
import numpy

In [10]:
numpy.__version__

'1.19.2'

In [11]:
from gensim import corpora, models, similarities
import gensim

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


为了免去讲解安装NLTK等等的麻烦，这里直接手写一下**停止词列表**：

这些词在不同语境中指代意义完全不同，但是在不同主题中的出现概率是几乎一致的。所以要去除，否则对模型的准确性有影响

In [12]:
stoplist = ['very', 'ourselves', 'am', 'doesn', 'through', 'me', 'against', 'up', 'just', 'her', 'ours', 
            'couldn', 'because', 'is', 'isn', 'it', 'only', 'in', 'such', 'too', 'mustn', 'under', 'their', 
            'if', 'to', 'my', 'himself', 'after', 'why', 'while', 'can', 'each', 'itself', 'his', 'all', 'once', 
            'herself', 'more', 'our', 'they', 'hasn', 'on', 'ma', 'them', 'its', 'where', 'did', 'll', 'you', 
            'didn', 'nor', 'as', 'now', 'before', 'those', 'yours', 'from', 'who', 'was', 'm', 'been', 'will', 
            'into', 'same', 'how', 'some', 'of', 'out', 'with', 's', 'being', 't', 'mightn', 'she', 'again', 'be', 
            'by', 'shan', 'have', 'yourselves', 'needn', 'and', 'are', 'o', 'these', 'further', 'most', 'yourself', 
            'having', 'aren', 'here', 'he', 'were', 'but', 'this', 'myself', 'own', 'we', 'so', 'i', 'does', 'both', 
            'when', 'between', 'd', 'had', 'the', 'y', 'has', 'down', 'off', 'than', 'haven', 'whom', 'wouldn', 
            'should', 've', 'over', 'themselves', 'few', 'then', 'hadn', 'what', 'until', 'won', 'no', 'about', 
            'any', 'that', 'for', 'shouldn', 'don', 'do', 'there', 'doing', 'an', 'or', 'ain', 'hers', 'wasn', 
            'weren', 'above', 'a', 'at', 'your', 'theirs', 'below', 'other', 'not', 're', 'him', 'during', 'which']

人工分词：

这里，英文的分词，直接就是对着空白处分割就可以了。

中文的分词稍微复杂点儿，具体可以百度：CoreNLP, HaNLP, 结巴分词，等等

分词的意义在于，把我们的长长的字符串原文本，转化成有意义的小元素：

In [13]:
texts = [[word for word in doc.lower().split() if word not in stoplist] for doc in doclist]

这时候，我们的texts就是我们需要的样子了：

In [14]:
texts[0]

['thursday',
 'march',
 'pm',
 'latest',
 'syria',
 'aiding',
 'qaddafi',
 'sid',
 'hrc',
 'memo',
 'syria',
 'aiding',
 'libya',
 'docx',
 'hrc',
 'memo',
 'syria',
 'aiding',
 'libya',
 'docx',
 'march',
 'hillary']

### 建立语料库

用词袋的方法，把每个单词用一个数字index指代，并把我们的原文本变成一条长长的数组：

In [15]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [16]:
corpus[13]

[(51, 1), (505, 1), (506, 1), (507, 1), (508, 1)]

这个列表告诉我们，第14（从0开始是第一）个邮件中，一共6个有意义的单词（经过我们的文本预处理，并去除了停止词后）

其中，36号单词出现1次，505号单词出现1次，以此类推。。。

接着，可以建立模型了：

In [17]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

我们可以看到，第10号分类，其中最常出现的单词是：

In [18]:
lda.print_topic(10, topn=5)

'0.013*"strategic" + 0.013*"afghan" + 0.012*"press" + 0.012*"doc" + 0.009*"dialogue"'

我们把所有的主题打印出来看看

In [19]:
lda.print_topics(num_topics=20, num_words=5)

[(0,
  '0.026*"pm" + 0.018*"taliban" + 0.010*"pis" + 0.010*"print" + 0.010*"declassify"'),
 (1,
  '0.016*"un" + 0.012*"pls" + 0.012*"print" + 0.008*"state" + 0.007*"development"'),
 (2,
  '0.014*"senate" + 0.013*"republicans" + 0.012*"percent" + 0.012*"health" + 0.008*"would"'),
 (3,
  '0.009*"sbwhoeop" + 0.007*"sid" + 0.005*"sunday" + 0.005*"us" + 0.005*"per"'),
 (4,
  '0.038*"call" + 0.017*"tomorrow" + 0.017*"see" + 0.015*"ok" + 0.013*"thx"'),
 (5,
  '0.016*"israeli" + 0.015*"settlements" + 0.014*"israel" + 0.007*"would" + 0.007*"miliband"'),
 (6,
  '0.017*"fw" + 0.015*"cheryl" + 0.014*"pm" + 0.013*"mills" + 0.008*"tuesday"'),
 (7,
  '0.013*"ok" + 0.012*"boehner" + 0.012*"party" + 0.010*"labour" + 0.008*"tax"'),
 (8,
  '0.010*"women" + 0.008*"haitian" + 0.008*"kurdistan" + 0.007*"one" + 0.006*"could"'),
 (9,
  '0.041*"yes" + 0.012*"eikenberry" + 0.008*"yep" + 0.008*"et" + 0.007*"coming"'),
 (10,
  '0.013*"strategic" + 0.013*"afghan" + 0.012*"press" + 0.012*"doc" + 0.009*"dialogue"'),

### 接下来：

通过
```
lda.get_document_topics(bow)
```
或者
```
lda.get_term_topics(word_id)
```

两个方法，我们可以把新鲜的文本/单词，分类成20个主题中的一个。

*但是注意，我们这里的文本和单词，都必须得经过同样步骤的文本预处理+词袋化，也就是说，变成数字表示每个单词的形式。*

